In [23]:
import pygame as pg

In [24]:
pg.mixer.init()
pg.init()

(5, 0)

In [25]:
def path(note):
    return "Piano/" + note + ".wav"

In [26]:
def play_note(note):
    pg.mixer.music.load(path(note))
    pg.mixer.music.play()

In [27]:
pg.mixer.set_num_channels(1)

In [30]:
play_note('fs4')